### Building a standard randomforest to use when playing around with feature selection techniques.

### Also, trying to figure out how to access the results from a script in a jupyter notebook.

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from project import handling_missing_values

This is me playing around with trying to clean the data outside of the jupyter notebook

In [2]:
data = !python clean_data.py train.csv

In [3]:
columns = data[0].split(',')

In [4]:
data = pd.DataFrame(data[1:])[0].str.split(',',expand=True)

In [5]:
data.columns = columns

In [6]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NoAlley,Reg,Lvl,AllPub,...,0,NoPool,NoFence,NoMisc,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NoAlley,Reg,Lvl,AllPub,...,0,NoPool,NoFence,NoMisc,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NoAlley,IR1,Lvl,AllPub,...,0,NoPool,NoFence,NoMisc,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NoAlley,IR1,Lvl,AllPub,...,0,NoPool,NoFence,NoMisc,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NoAlley,IR1,Lvl,AllPub,...,0,NoPool,NoFence,NoMisc,0,12,2008,WD,Normal,250000


In [7]:
data_types = ['int64','int64','object','float64','int64','object','object','object','object','object','object',
              'object','object','object','object','object','object','int64','int64','int64','int64','object',
              'object','object','object','object','float64','object','object','object','object','object','object',
              'object','int64','object','int64','int64','int64','object','object','object','object','int64','int64',
              'int64','int64','int64','int64','int64','int64','int64','int64','object','int64','object','int64',
              'object','object','float64','object','int64','int64','object','object','object','int64','int64',
              'int64','int64','int64','int64','object','object','object','int64','int64','int64','object','object',
              'int64']

In [8]:
for col, dtype in zip(columns, data_types):
    data[col] = data[col].astype(dtype)

now lets package the restoring of the data into a nice function

In [9]:
def restoring_data(data):
    columns = data[0].split(',')
    data = pd.DataFrame(data[1:])[0].str.split(',',expand=True)
    data.columns = columns
    for col, dtype in zip(columns, data_types):
        data[col] = data[col].astype(dtype)
    return data

In [10]:
data = !python clean_data.py train.csv

In [11]:
data = restoring_data(data)

In [12]:
data.isnull().any().any()

False

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 81 columns):
Id               1459 non-null int64
MSSubClass       1459 non-null int64
MSZoning         1459 non-null object
LotFrontage      1459 non-null float64
LotArea          1459 non-null int64
Street           1459 non-null object
Alley            1459 non-null object
LotShape         1459 non-null object
LandContour      1459 non-null object
Utilities        1459 non-null object
LotConfig        1459 non-null object
LandSlope        1459 non-null object
Neighborhood     1459 non-null object
Condition1       1459 non-null object
Condition2       1459 non-null object
BldgType         1459 non-null object
HouseStyle       1459 non-null object
OverallQual      1459 non-null int64
OverallCond      1459 non-null int64
YearBuilt        1459 non-null int64
YearRemodAdd     1459 non-null int64
RoofStyle        1459 non-null object
RoofMatl         1459 non-null object
Exterior1st      1459 non

### Back to a random forest

In [14]:
data = pd.read_csv('train.csv')

In [15]:
handling_missing_values(data)

In [16]:
target = [0 if price <= 120000 else 1 if price <= 200000 else 2 for price in data.SalePrice]

In [17]:
data.drop(['Id','SalePrice'], axis=1, inplace=True)

In [18]:
data = data.astype({'LotFrontage': 'int64', 'MasVnrArea': 'int64', 'GarageYrBlt': 'int64'})

Have to convert categorical variables

2 things need to be done:
1. onehot encoding for categorical variables
2. turn ordinal data into numeric

In [19]:
categorical_data = ['MSSubClass', 'MSZoning', 'Alley', 'LandContour', 'LotConfig',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'Foundation', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'Functional', 'GarageType',
       'Fence', 'MiscFeature', 'YrSold', 'SaleType', 'SaleCondition']

In [20]:
numeric_data = ['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
                'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
                'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
                'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt',
                'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
                'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold']

In [21]:
boolean_data = ['Street', 'CentralAir']

just going to enconde boolean data using the ordinal encoder = same results

In [22]:
ordinal_data = ['LotShape', 'Utilities', 'LandSlope', 'ExterQual', 'ExterCond', 'BsmtQual',
                'BsmtCond', 'BsmtExposure', 'HeatingQC', 'Electrical', 'KitchenQual', 'FireplaceQu',
                'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC']

In [23]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', dtype='int8'), categorical_data),
        ('ord', OrdinalEncoder(dtype='int8'), ordinal_data+boolean_data),
        ('num', 'passthrough', numeric_data)])

In [24]:
transformed_data = preprocessor.fit_transform(data)

In [25]:
transformed_data.shape

(1459, 259)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(transformed_data, target)

In [27]:
rf = RandomForestClassifier()

In [28]:
rf.fit(X_train, y_train)

/Users/tovaschwartz/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [29]:
rf.score(X_train, y_train)

0.9945155393053017

In [30]:
rf.score(X_test, y_test)

0.8136986301369863

Playing around to make sure the transformed data makes sense

In [31]:
data.tail()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1455,60,RL,62,7917,Pave,NoAlley,Reg,Lvl,AllPub,Inside,...,0,0,NoPool,NoFence,NoMisc,0,8,2007,WD,Normal
1456,20,RL,85,13175,Pave,NoAlley,Reg,Lvl,AllPub,Inside,...,0,0,NoPool,MnPrv,NoMisc,0,2,2010,WD,Normal
1457,70,RL,66,9042,Pave,NoAlley,Reg,Lvl,AllPub,Inside,...,0,0,NoPool,GdPrv,Shed,2500,5,2010,WD,Normal
1458,20,RL,68,9717,Pave,NoAlley,Reg,Lvl,AllPub,Inside,...,0,0,NoPool,NoFence,NoMisc,0,4,2010,WD,Normal
1459,20,RL,75,9937,Pave,NoAlley,Reg,Lvl,AllPub,Inside,...,0,0,NoPool,NoFence,NoMisc,0,6,2008,WD,Normal


In [32]:
transformed_data[0,:]

array([   0,    0,    0,    0,    0,    1,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    1,    0,    0,    1,
          0,    0,    0,    0,    1,    0,    0,    0,    0,    1,    0,
          0,    0,    0,    0,    1,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    1,    0,    0,    0,    0,    0,    0,
          0,    0,    1,    0,    0,    0,    0,    0,    1,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    1,    0,    0,    0,
          1,    0,    0,    0,    0,    0,    1,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    1,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    1,    0,    0,
          0,    1,    0,    0,    0,    0,    1,    0,    0,    0,    0,
          0,    1,    0,    0,    0,    0,    0,   

### Putting it all together into a function

In [33]:
def classification_target(data):
    return [0 if price <= 120000 else 1 if price <= 200000 else 2 for price in data['SalePrice']]

In [34]:
def preprocessed_data(data):
    """returns preprocessed data as numpy array ready for decision tree"""
    data.drop(['Id','SalePrice'], axis=1, inplace=True)
    data.astype({'LotFrontage': 'int64', 'MasVnrArea': 'int64', 'GarageYrBlt': 'int64'}, inplace=True)
    
    categorical_data = ['MSSubClass', 'MSZoning', 'Alley', 'LandContour', 'LotConfig',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'Foundation', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'Functional', 'GarageType',
       'Fence', 'MiscFeature', 'YrSold', 'SaleType', 'SaleCondition']
    
    numeric_data = ['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
                'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
                'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
                'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt',
                'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
                'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold']
    
    boolean_data = ['Street', 'CentralAir']
    
    ordinal_data = ['LotShape', 'Utilities', 'LandSlope', 'ExterQual', 'ExterCond', 'BsmtQual',
                'BsmtCond', 'BsmtExposure', 'HeatingQC', 'Electrical', 'KitchenQual', 'FireplaceQu',
                'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC']
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(handle_unknown='ignore', dtype='int8'), categorical_data),
            ('ord', OrdinalEncoder(dtype='int8'), ordinal_data+boolean_data),
            ('num', 'passthrough', numeric_data)])
    
    return preprocessor.fit_transform(data)

In [35]:
data = pd.read_csv('train.csv')

In [36]:
handling_missing_values(data)

In [37]:
target = classification_target(data)

In [38]:
len(target)

1459

In [39]:
transformed_data = preprocessed_data(data)

In [40]:
transformed_data.shape

(1459, 259)

In [41]:
X_train, X_test, y_train, y_test = train_test_split(transformed_data, target)

In [42]:
rf = RandomForestClassifier()

In [43]:
rf.fit(X_train, y_train)

/Users/tovaschwartz/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [44]:
rf.score(X_train, y_train)

0.9990859232175503

In [45]:
rf.score(X_test, y_test)

0.7616438356164383